# Concurrency ⚡️

### Threads / async / multiprocessing

### Asincronía vs paralelismo

* enviar/recibir datos a través de la red
* leer el contenido de un archivo dentro de nuestro programa
* escribir datos de nuestro programa en el disco
* esperar a que finalice una operación en una API remota
* esperar a que finalice una operación en una base de datos 
* etc.

### Que puede ser paralelizado y que no

Cuando llegamos a los detalles, solo el `multiprocessing` realmente ejecuta estos hilos de procesamiento literalmente al mismo tiempo.

`threading` y `asyncio` se ejecutan en un único proceso y, por lo tanto, solo se ejecutan uno a la vez. Simplemente encuentran formas de turnarse para acelerar el proceso general. Aún así llamamos a esto concurrencia.

* threading: multitarea apropiativa (OS. El SO decide.) https://es.wikipedia.org/wiki/Multitarea_apropiativa  
* asyncio: multitarea cooperativa (Tu. Cada proceso cede el control.) https://es.wikipedia.org/wiki/Multitarea_cooperativa

### Memoria compartida vs replicación

Cosas más lentas que la CPU; I/O o network bound.

![I/O - networking](https://files.realpython.com/media/IOBound.4810a888b457.png)

No asociado a I/O, mucha computación; CPU bound.

![](https://files.realpython.com/media/CPUBound.d2d32cb2626c.png)

In [1]:
sites = [
    "https://www.yahoo.com/",
    "http://www.cnn.com",
    "http://www.python.org",
    "http://www.jython.org",
    "http://www.pypy.org",
    "http://www.perl.org",
    "http://www.cisco.com",
    "http://www.facebook.com",
    "http://www.twitter.com",
    "http://www.macrumors.com/",
    "http://arstechnica.com/",
    "http://www.reuters.com/",
    "http://abcnews.go.com/",
    "http://www.cnbc.com/",
    "http://olympus.realpython.org/dice",
    "https://realpython.com/",
]

In [2]:
import requests
import time


def download_site(url, session):
    with session.get(url) as response:
        print(f"Read {len(response.content)} from {url}")


def download_all_sites(sites):
    with requests.Session() as session:
        for url in sites:
            download_site(url, session)


start_time = time.time()
download_all_sites(sites)
duration = time.time() - start_time
print(duration)

Read 244295 from https://www.yahoo.com/
Read 1142002 from http://www.cnn.com
Read 49886 from http://www.python.org
Read 10394 from http://www.jython.org
Read 6832 from http://www.pypy.org
Read 12861 from http://www.perl.org
Read 96543 from http://www.cisco.com
Read 209853 from http://www.facebook.com
Read 42523 from http://www.twitter.com
Read 336513 from http://www.macrumors.com/
Read 90569 from http://arstechnica.com/
Read 203818 from http://www.reuters.com/
Read 212963 from http://abcnews.go.com/
Read 971874 from http://www.cnbc.com/
Read 276 from http://olympus.realpython.org/dice
Read 39789 from https://realpython.com/
12.013489007949829


### Threading

In [3]:
from concurrent.futures import ThreadPoolExecutor
import requests
import threading
import time

# Threading.local() creates an object that look like a global but is specific to each individual thread.
thread_local = threading.local()


# requests.Session() is not thread-safe
def get_session():
    if not hasattr(thread_local, "session"):
        thread_local.session = requests.Session()
    return thread_local.session


def download_site(url):
    session = get_session()
    with session.get(url) as response:
        print(f"Read {len(response.content)} from {url}")


def download_all_sites(sites):
    # with -> create and free thread pool
    with ThreadPoolExecutor(max_workers=5) as executor:
        executor.map(download_site, sites)


start_time = time.time()
download_all_sites(sites)
duration = time.time() - start_time
print(duration)

Read 10394 from http://www.jython.org
Read 49886 from http://www.python.org
Read 1142002 from http://www.cnn.com
Read 6832 from http://www.pypy.org
Read 96543 from http://www.cisco.com
Read 12861 from http://www.perl.org
Read 336467 from http://www.macrumors.com/
Read 206758 from http://www.facebook.com
Read 204144 from http://www.reuters.com/
Read 245526 from https://www.yahoo.com/
Read 90569 from http://arstechnica.com/
Read 42523 from http://www.twitter.com
Read 276 from http://olympus.realpython.org/dice
Read 970282 from http://www.cnbc.com/
Read 212963 from http://abcnews.go.com/
Read 39789 from https://realpython.com/
2.04518723487854


La parte del `Thread`. Eso es solo un hilo de procesamiento que mencionamos anteriormente. `Pool` es donde comienza a ponerse interesante. Este objeto va a crear un grupo de subprocesos, cada uno de los cuales puede ejecutarse simultáneamente. Finalmente, el `Executor` es la parte que controlará cómo y cuándo se ejecutará cada uno de los hilos del grupo.

![](https://files.realpython.com/media/Threading.3eef48da829e.png)

In [4]:
5000 * 100

500000

In [ ]:
with open("sadasd.ttxt", "r") as f:
    texto = f.read()

In [48]:
class Lock:
    
    def __init__(self, name):
        self.name = name
        self.acquired = False
        
    def acquire(self):
        self.acquired = True
        return True
    
    def __enter__(self):
        try:
            self.acquire()
        finally:
            self.release()
        print("entrando")
        return self
        
    def __exit__(self):
        print("saliendi")
        return self

In [47]:
import concurrent.futures
# from threading import Lock


counter = 0

l = Lock()

def increment_counter(fake_value):
    # l.acquire()
    with l:
        global counter
        for _ in range(100):
            counter += 1
    # l.release()


fake_data = [x for x in range(5000)]

# counter = 0

with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
    executor.map(increment_counter, fake_data)

In [28]:
print(counter)

500000


### Async

The general concept of asyncio is that a single Python object, called the event loop, controls how and when each task gets run. The event loop is aware of each task and knows what state it’s in. In reality, there are many states that tasks could be in, but for now let’s imagine a simplified event loop that just has two states.

The ready state will indicate that a task has work to do and is ready to be run, and the waiting state means that the task is waiting for some external thing to finish, such as a network operation.

Your simplified event loop maintains two lists of tasks, one for each of these states. It selects one of the ready tasks and starts it back to running. That task is in complete control until it cooperatively hands the control back to the event loop.

When the running task gives control back to the event loop, the event loop places that task into either the ready or waiting list and then goes through each of the tasks in the waiting list to see if it has become ready by an I/O operation completing. It knows that the tasks in the ready list are still ready because it knows they haven’t run yet.

Once all of the tasks have been sorted into the right list again, the event loop picks the next task to run, and the process repeats. Your simplified event loop picks the task that has been waiting the longest and runs that. This process repeats until the event loop is finished.

An important point of asyncio is that the tasks never give up control without intentionally doing so. They never get interrupted in the middle of an operation. This allows us to share resources a bit more easily in asyncio than in threading. You don’t have to worry about making your code thread-safe.

**Any function that calls await needs to be marked with async. You’ll get a syntax error otherwise.**

No hay que preocuparse del número de threads que crear

https://markhneedham.com/blog/2019/05/10/jupyter-runtimeerror-this-event-loop-is-already-running/

In [51]:
# if __name__ == "__main__":
#     asyncio.run(download_sites)

In [53]:
import asyncio
import time
import aiohttp


async def download_site(session, url):
    async with session.get(url) as response:
        # result = await response.content_length
        print("Read {0} from {1}".format(response.content_length, url))


async def download_all_sites(sites):
    # You can share the session across all tasks, so the session is created here as a context manager.
    # The tasks can share the session because they are all running on the same thread.
    async with aiohttp.ClientSession() as session:
        tasks = []
        for url in sites:
            # ensure_future also takes care of starting the tasks
            task = asyncio.ensure_future(download_site(session, url))
            tasks.append(task)
        # print(tasks)
        # Once all the tasks are created, this function uses asyncio.gather()
        # to keep the session context alive until all of the tasks have completed.
        await asyncio.gather(*tasks, return_exceptions=True)


start_time = time.time()

await download_all_sites(sites)

duration = time.time() - start_time
print(f"Downloaded {len(sites)} sites in {duration} seconds")

Read 45718 from http://www.reuters.com/
Read 12861 from http://www.perl.org
Read 49886 from http://www.python.org
Read 3586 from http://www.jython.org
Read 153581 from http://www.cnbc.com/
Read 20418 from http://www.cisco.com
Read 276 from http://olympus.realpython.org/dice
Read None from http://www.macrumors.com/
Read None from https://realpython.com/
Read 157341 from http://www.cnn.com
Read None from http://www.facebook.com
Read 6832 from http://www.pypy.org
Read 50102 from http://abcnews.go.com/
Read None from http://arstechnica.com/
Read None from http://www.twitter.com
Read None from https://www.yahoo.com/
Downloaded 16 sites in 1.0771641731262207 seconds


![](https://files.realpython.com/media/Asyncio.31182d3731cf.png)

**ATENCIÓN**

**ATENCIÓN**

**ATENCIÓN**

Las siguientes celdas demuestran como descargar un set de imágenes aleatorias de la API de [unsplash](https://unsplash.com/). Una de las celdas las escribe en una carpeta `images/` la otra en `images2/`.


En la carpeta donde está este notebook están estas dos carpetas vacias. Pero revisa todo bien para que no se sobreescriba nada que no quieras.

Las celdas están aquí para demostrar como descargar muchas imágenes de forma asíncrona usando [aiohtpp](https://docs.aiohttp.org/en/stable/) en el primer caso, y [httpx](https://github.com/encode/httpx) en el segundo.

In [6]:
from itertools import repeat

from string import ascii_lowercase, printable
from random import choice


def random_string(string_length=15):
    """Generate a random string of fixed length """
    letters = ascii_lowercase
    return "".join(choice(letters) for i in range(string_length))


import asyncio
import aiohttp
import aiofiles


async def download_site(session, url):
    async with session.get(url) as response:
        c = await response.read()
        await write_file(c)


async def write_file(content):

    filename = "images/" + random_string() + ".jpg"
    async with aiofiles.open(filename, mode="wb") as f:
        await f.write(content)


urls = [
    "https://source.unsplash.com/1600x900/?nature,water," + random_string(6)
    for _ in range(50)
]


async def dl():
    async with aiohttp.ClientSession() as session:
        tasks = []
        for url in urls:
            task = asyncio.ensure_future(download_site(session, url))
            tasks.append(task)

        await asyncio.gather(*tasks, return_exceptions=True)


start = time.perf_counter()
await dl()
print(f"total = {time.perf_counter() - start}")

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 22.2 µs
total = 4.9483451249998325


In [54]:
from itertools import repeat

from string import ascii_lowercase
from random import choice


def random_string(string_length=15):
    """Generate a random string of fixed length """
    letters = ascii_lowercase
    return "".join(choice(letters) for i in range(string_length))


import asyncio
import time
import httpx
import aiofiles


async def download_site(client, url):
    r = await client.get(url)
    await write_file(r.content)


async def write_file(content):

    filename = "images2/" + random_string() + ".jpg"
    async with aiofiles.open(filename, mode="wb") as f:
        await f.write(content)


urls = [
    "https://source.unsplash.com/1600x900/?nature,water," + random_string(10)
    for _ in range(50)
]


async def dl():
    async with httpx.AsyncClient() as client:
        tasks = []
        for url in urls:
            task = asyncio.ensure_future(download_site(client, url))
            tasks.append(task)

        await asyncio.gather(*tasks, return_exceptions=True)


start = time.perf_counter()
await dl()
print(f"total = {time.perf_counter() - start}")

total = 6.267024283999945


Ejercicio opcional, leer y entender este código: https://pybay.com/site_media/slides/raymond2017-keynote/async_examples.html

In [5]:
# adapted from https://gist.github.com/bradmontgomery/81d71e415b0ff693f00408388590acb9

import hashlib
import sys

from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from time import sleep, time


def t1(n):
    """Silly function whose time increases as n does, it increases linearly."""
    for i in range(n):
        if i % 2 == 0:
            sleep(0.5)


def t2(n):
    """A somewhat CPU-intensive task."""
    for i in range(n):
        hashlib.pbkdf2_hmac("sha256", b"password", b"salt", 100000)


def do_work(n):
    """Function that does t1 and t2 in serial."""
    start = time()
    t1(n)
    t2(n)
    end = time()
    print("Work for {} finished in {}s".format(n, round(end - start, 2)))

In [6]:
def serial():

    start = time()
    for x in range(10):
        do_work(x)
    end = time()
    print("All work finished in {}s".format(round(end - start, 2)))

In [7]:
def parallel():
    start = time()
    with ThreadPoolExecutor(max_workers=4) as executor:
        inputs = range(10)
        for x, result in zip(inputs, executor.map(do_work, inputs)):
            pass
    end = time()
    print("All work finished in {}s".format(round(end - start, 2)))

In [27]:
serial()

Work for 0 finished in 0.0s
Work for 1 finished in 0.59s
Work for 2 finished in 0.66s
Work for 3 finished in 1.26s
Work for 4 finished in 1.4s
Work for 5 finished in 1.93s
Work for 6 finished in 2.05s
Work for 7 finished in 2.58s
Work for 8 finished in 2.69s
Work for 9 finished in 3.25s
All work finished in 16.42s


In [8]:
parallel()

Work for 0 finished in 0.0s
Work for 1 finished in 0.63s
Work for 2 finished in 0.78s
Work for 3 finished in 1.32s
Work for 4 finished in 1.4s
Work for 5 finished in 1.9s
Work for 6 finished in 2.36s
Work for 7 finished in 2.75s
Work for 8 finished in 2.79s
Work for 9 finished in 3.22s
All work finished in 5.75s


### Nº

In [9]:
import multiprocessing as mp

print("Number of processors: ", mp.cpu_count())

Number of processors:  4


In [10]:
import os

os.cpu_count()

4

### Ejercicio

* Escribe un script que identifique todas las imágenes de un árbol de carpetas.
* Debemos obtener una lista con todas las rutas de archivo de las imagénes.
* Crear una función que covierta una imagen a 128x128. Usar la librería **Pillow**, ya viene instalada en vuestra distribución de Anaconda creo.

```python
import PIL
````

* Tras convertir una imagen, todas deben estar guardadaes en una misma carpeta. Por ejemplo al final habrá una carpeta que se llame "miniaturas" que contendrá todas las imágenes convertidas.
* Cada imagen debe convertirla en un thumbnail (128x128) y guardarlas en una misma carpeta.
* Cuando guardemos la imagen debemos guardarla con su nombre original añadiendo "_thumbnail".
    Por ejemplo `imagen.jpg` -> `imagen_thumbnail.jpg`

Intentar usar un f-string para el path `(f"carpeta/{}_{}.jpg")`.

* **Importante**: una vez tengamos la lista con todas nuestras rutas de archivo. Hay que usar procesamiento en paralelo para convertir las imágenes. Por ejemplo un ThreadPoolExecutor o ProcessPoolExecutor.

**Extra**:

En el módulo `functools` de Python existe una cosa que se llama `partial`. Esta función nos permite crea lo que se llaman funciones parciales. Si tenemos una función que por ejemplo acepta 3 argumentos, crear una función parcial sería *"duplicar"* está función pero haciendo que uno de estos parámetros sea fijo. Y obtendríamos una función. Por ejemplo:

* Tengo una función: `convertir_miniatura(resolucion, ruta)`
* Puedo hacer `miniatura128 = partial(convertir_miniatura, 128)`.
* Esto último me devolvería otra función, que ahora puedo utilizar directamente con: `miniatura128("/Users/r/.../imagen.jpg")`. Tendremos a nuestra disposición una nueva función que es igual que la original pero como si uno de sus parámetros estuviera fijo.


`functools.partial` + executors Pillow + paths (download images)




Ejercicio adaptado de: https://www.toptal.com/python/beginners-guide-to-concurrency-and-parallelism-in-python

### Ejercicio pistas

In [ ]:
def reducir_imagen(ruta):
    img =Image.open(ruta)
    img.resize
    img.save("")

In [46]:
import os

filelist = []

for _, _, _ in os.walk("tree"):
    # os.walk itera sobre 3 parámetros, cuales son?
    # hacer algo
    pass

In [11]:
from pathlib import Path

In [14]:
p = Path("tree/fdjtoupvvurxgrd.jpg")

In [15]:
p.absolute()  # <<-- es una función, hay que poner ()

PosixPath('/Users/r/Projects/teach/11_concurrencia_paralelismo/tree/fdjtoupvvurxgrd.jpg')

In [16]:
p.name  # es un método, NO hay que poner ()

'fdjtoupvvurxgrd.jpg'

In [17]:
p.stem  # es un método, NO hay que poner ()

'fdjtoupvvurxgrd'

In [18]:
p.suffix

'.jpg'

In [19]:
nuevo_nombre = p.stem + "_thumbnail" + p.suffix

In [21]:
miniaturas = Path("miniaturas/")

In [22]:
miniaturas/nuevo_nombre

PosixPath('miniaturas/fdjtoupvvurxgrd_thumbnail.jpg')

In [4]:
import os
from pathlib import Path

In [5]:
lista_de_paths = []
for root, dirs, files in os.walk("tree"):
    lista_de_paths = lista_de_paths + [os.path.join(root, x) for x in files if x.endswith((".jpg"))]

In [6]:
[str(ruta.absolute()) for ruta in Path("tree").rglob("*.jpg")]

['/Users/r/Projects/teach/11_concurrencia_paralelismo/tree/jkrlyxsbonbrdjb.jpg',
 '/Users/r/Projects/teach/11_concurrencia_paralelismo/tree/qynjddmvbflfmfl.jpg',
 '/Users/r/Projects/teach/11_concurrencia_paralelismo/tree/pjeyfssssqovicw.jpg',
 '/Users/r/Projects/teach/11_concurrencia_paralelismo/tree/rvqjyhrsggnkgmf.jpg',
 '/Users/r/Projects/teach/11_concurrencia_paralelismo/tree/mtshvhafwuveczn.jpg',
 '/Users/r/Projects/teach/11_concurrencia_paralelismo/tree/hriejwwwxnlslnr.jpg',
 '/Users/r/Projects/teach/11_concurrencia_paralelismo/tree/ufufkvakojndran.jpg',
 '/Users/r/Projects/teach/11_concurrencia_paralelismo/tree/rxvjgoudtzlgska.jpg',
 '/Users/r/Projects/teach/11_concurrencia_paralelismo/tree/fdjtoupvvurxgrd.jpg',
 '/Users/r/Projects/teach/11_concurrencia_paralelismo/tree/tjbcluxsujxgcnw.jpg',
 '/Users/r/Projects/teach/11_concurrencia_paralelismo/tree/msqfostmedkgorc.jpg',
 '/Users/r/Projects/teach/11_concurrencia_paralelismo/tree/E5E2Q06R/ogdottndbujeyrq.jpg',
 '/Users/r/Projects

In [7]:
filelist = lista_de_paths.copy()

In [10]:
from pathlib import Path
import PIL
from PIL import Image
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

filelist = [str(ruta.absolute()) for ruta in Path("tree/").rglob("*jpg")]


def miniaturizar(path: str):
    size = (128, 128)  # 128x128
    p = Path(path).absolute()
    nuevo_nombre = p.stem + "_thumbnail" + p.suffix
    miniaturas = Path("miniaturas/").absolute()
    save = miniaturas / nuevo_nombre
    image = Image.open(p)
    image.thumbnail(size)
    image.save(save)



import os
from joblib import Parallel, delayed

max_nucleos = os.cpu_count()

In [19]:
%%time
Parallel(n_jobs=max_nucleos, verbose=10)(delayed(miniaturizar)(p) for p in filelist)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    2.1s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    2.2s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:    2.3s
[Parallel(n_jobs=4)]: Batch computation too fast (0.1837s.) Setting batch_size=2.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    2.4s
[Parallel(n_jobs=4)]: Done  34 tasks      | elapsed:    2.7s
[Parallel(n_jobs=4)]: Batch computation too fast (0.1563s.) Setting batch_size=4.
[Parallel(n_jobs=4)]: Done  56 tasks      | elapsed:    3.1s


CPU times: user 121 ms, sys: 55.7 ms, total: 177 ms
Wall time: 3.94 s


[Parallel(n_jobs=4)]: Done  94 tasks      | elapsed:    3.8s
[Parallel(n_jobs=4)]: Done  97 out of 104 | elapsed:    3.9s remaining:    0.3s
[Parallel(n_jobs=4)]: Done 104 out of 104 | elapsed:    3.9s finished


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [17]:
%%time
with ThreadPoolExecutor(max_workers=max_nucleos) as executor:
    executor.map(miniaturizar, filelist)

CPU times: user 1.14 s, sys: 131 ms, total: 1.27 s
Wall time: 809 ms


In [16]:
%%time
for ruta in filelist:
    miniaturizar(ruta)

CPU times: user 927 ms, sys: 77.5 ms, total: 1 s
Wall time: 1.31 s


In [34]:
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
import os

In [35]:
%time


# with ProcessPoolExecutor(max_workers=os.cpu_count()) as executor:
# nº maximo de cpus lo calcula solo ^^^^^^^^^^^^^
with ThreadPoolExecutor(max_workers=4) as executor:
    executor.map(miniaturizar, filelist)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 4.77 µs


### Alternativa

Usando una [función parcial](https://docs.python.org/3/library/functools.html#functools.partial)

In [2]:
import os

In [3]:
filelist = []
for root, dirs, files in os.walk("res"):
    filelist = filelist + [os.path.join(root, x) for x in files if x.endswith((".jpg"))]

In [36]:
import logging
from pathlib import Path
from time import time
from functools import partial

from concurrent.futures import ProcessPoolExecutor

from PIL import Image

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

logger = logging.getLogger(__name__)


def create_thumbnail(img, size, save):
    """
    Creates a thumbnail of an image with the same name as image but with
    _thumbnail appended before the extension. E.g.:

    >>> create_thumbnail((128, 128), 'image.jpg')

    A new thumbnail image is created with the name image_thumbnail.jpg

    :param size: A tuple of the width and height of the image
    :param path: The path to the image file
    :return: None
    """
    path = img
    print("Current image:", path)
    print("Save path:", save)
    print("Size:", size)
    path = Path(path)
    name = path.stem + "_thumbnail" + path.suffix
    thumbnail_path = Path(save) / name
    image = Image.open(path)
    image.thumbnail(size)
    image.save(thumbnail_path)

In [37]:
thumbnail_128 = partial(create_thumbnail, size=(128, 128), save="save")

In [ ]:
# aplicado a una image
thumbnail_128(img=filelist[0])

In [41]:
ts = time()
# Partially apply the create_thumbnail method, setting the size to 128x128
# and returning a function of a single argument.


# Create the executor in a with block so shutdown is called when the block
# is exited.
with ProcessPoolExecutor() as executor:
    executor.map(thumbnail_128, filelist)


logging.info("Took %s", time() - ts)

2020-10-27 14:02:46,893 - root - INFO - Took 0.5612080097198486


[Donald Knuth](https://en.wikipedia.org/wiki/Donald_Knuth): “Premature optimization is the root of all evil (or at least most of it) in programming.”

Más info y fuentes:

* https://realpython.com/python-concurrency/ 👈🏼
* https://realpython.com/intro-to-python-threading/
* https://www.youtube.com/watch?v=9zinZmE3Ogk
* https://pybay.com/site_media/slides/raymond2017-keynote/index.html
* https://realpython.com/async-io-python/  👈🏼 lectura recomendada, async es un tema complejo y tiene su curva de apredizaje
* https://realpython.com/intro-to-python-threading/
* https://www.toptal.com/python/beginners-guide-to-concurrency-and-parallelism-in-python
* https://stackoverflow.com/questions/49005651/how-does-asyncio-actually-work/51116910#51116910
* https://www.blog.pythonlibrary.org/2016/07/26/python-3-an-intro-to-asyncio/
* https://stackabuse.com/python-async-await-tutorial/